In [9]:
import os

In [10]:
os.chdir(r"C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow")

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path 
    unzip_dir: Path   

In [12]:
#importing all project paths from the constants file common action functions
from mlProject.constants import  *
from mlProject.utils.common import read_yaml, create_directories


In [13]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [14]:
import os
from urllib import request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


In [15]:
#Unpdate Components step
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the folloeing info: n\{headers}")
        else: 
            logger.info(f"File already exists: {get_size(Path(self.config.local_data_file))}")
        print("Downloding", self.config.source_url)
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts zip file into directory 
        Function returne none
        
        """
        
        unzip_path = self.config.unzip_dir
        # Print the actual path of the local data file and unzip directory
        print(f"Extracting file from: {self.config.local_data_file}")
        print(f"Unzip path: {unzip_path}")

        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [16]:
#Update pipeline Step 
try: 
    config = ConfigurationManager()
    data_ingestion_config =config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-13 14:15:00,539: INFO: common: yaml file: C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow\config\config.yaml loaded successfully]
[2025-05-13 14:15:00,548: INFO: common: yaml file: C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow\params.yaml loaded successfully]
[2025-05-13 14:15:00,564: INFO: common: yaml file: C:\Users\USER\Desktop\MLDefaults\End-to-End-Machine-Learning-with-Mlflow\schema.yaml loaded successfully]
[2025-05-13 14:15:00,568: INFO: common: created directory at:artifacts]
[2025-05-13 14:15:00,572: INFO: common: created directory at:artifacts/data_ingestion]
[2025-05-13 14:15:00,575: INFO: 292618914: File already exists: ~23 KB]
Downloding https://github.com/samaTech-sys/End-to-End-Machine-Learning-with-Mlflow/raw/main/winequality-red.zip
Extracting file from: artifacts/data_ingestion/data.zip
Unzip path: artifacts/data_ingestion
